## Here we create out training and validation sets

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import os
import shutil
from tqdm import tqdm
import math
from PIL import Image

In [2]:
from pylab import rcParams

rcParams['figure.figsize'] = 18, 10
sn.set_style('darkgrid')

In [4]:
train = pd.read_csv('data/myntra_train_dataset_with_fnames.csv')

In [5]:
# The training CSV has been preprocessed to include a column for filenames
# This would both help in moving files on the disk and also matching the 
# predictions to the file names later
train.head()

,Brand,Category,Gender,Color,Link_to_the_image,Sub_category,filename
0,Roadster,Tshirts,Men,Black,http://myntra.myntassets.com/assets/images/182...,Graphic,11501068910736-Roadster-Men-Black-Printed-Roun...
1,Roadster,Tshirts,Men,Black,http://myntra.myntassets.com/assets/images/182...,Graphic,11501754749483-Roadster-Men-Black-Printed-Roun...
2,Roadster,Tshirts,Men,Black,http://myntra.myntassets.com/assets/images/182...,Biker,11504765958740-Roadster-Men-Black-Printed-Henl...
3,Roadster,Tshirts,Men,Black,http://myntra.myntassets.com/assets/images/182...,Graphic,11501067533832-Roadster-Men-Black-Printed-Roun...
4,Roadster,Tshirts,Men,Black,http://myntra.myntassets.com/assets/images/182...,Graphic,11501067504556-Roadster-Men-Black-Printed-Roun...


In [24]:
# Move the images to the respective folder (class categories) to make the 
# creation of stratified validation split easier
BASE_PATH = 'data/train/'
SOURCE_BASE = 'data/images/'
for row in tqdm(train.iterrows()):
    sub_cat = row[1]['Sub_category']
    fname = row[1]['filename']
    fullpath = os.path.join(BASE_PATH, sub_cat)
    if not os.path.exists(fullpath):
        os.makedirs(fullpath)
    if isinstance(fname, str) and len(fname.strip()) > 0:
        srcpath = os.path.join(SOURCE_BASE, fname)
        if os.path.exists(srcpath):
            shutil.move(srcpath, fullpath)

70000it [00:32, 2173.35it/s]


In [30]:
for root, subdir, filenames in os.walk('data/train/'):
    for fname in tqdm(filenames):
        impath = os.path.join(root, fname)
        try:
            im = Image.open(impath)
        except:
            print('deleted {}'.format(impath))
            os.remove(impath)

0it [00:00, ?it/s]
  4%|▎         | 89/2510 [00:00<00:08, 294.40it/s]

deleted data/train/Colourblocked/_M8A2284_03.JPG?dl=1


 25%|██▍       | 621/2510 [00:11<00:36, 52.19it/s] 

deleted data/train/Colourblocked/83272601_1.jpg?dl=1


  0%|          | 26/6314 [00:00<00:51, 121.88it/s]

...

 19%|█▉        | 2487/13049 [00:31<02:13, 78.94it/s]

deleted data/train/Typography/59232101_1.jpg?dl=1


 43%|████▎     | 5633/13049 [01:08<01:30, 81.80it/s]

deleted data/train/Typography/MYNTRA_1.jpg?dl=1


100%|██████████| 608/608 [00:11<00:00, 53.11it/s]


In [31]:
TRAIN_DIR = 'data/images_train/train/'
VAL_DIR = 'data/images_train/val/'

In [45]:
# Create a validation set comprising 20% of training data.
import random
VAL_SIZE = 0.2
for subdir in os.listdir(TRAIN_DIR):
    trainpath = os.path.join(TRAIN_DIR, subdir)
    valpath = os.path.join(VAL_DIR, subdir)
    if not os.path.exists(valpath):
        os.mkdir(valpath)
    files = os.listdir(trainpath)
    size = int(VAL_SIZE * len(files))
    fchoices = random.choices(files, k=size)
    for f in tqdm(fchoices):
        try:
            shutil.move(os.path.join(trainpath, f), os.path.join(valpath, f))
        except:
            print('Error Moving', os.path.join(trainpath, f))
#         print('Moving', os.path.join(trainpath, f), 'to', os.path.join(valpath, f))

  0%|          | 0/501 [00:00<?, ?it/s]

Error Moving data/images_train/train/Abstract/11512799153171-Firetrap-Men-Khaki-Solid-Round-Neck-T-shirt-661512799153109-1.jpg
Error Moving data/images_train/train/Abstract/11451990237909-Jazzup-Pack-of-2-Printed-Slim-Fit-T-shirts-1271451990237408-1.jpg
Error Moving data/images_train/train/Abstract/11512799153171-Firetrap-Men-Khaki-Solid-Round-Neck-T-shirt-661512799153109-1.jpg
Error Moving data/images_train/train/Abstract/11493013808802-Hypernation-Women-Tshirts-51493013808598-1.jpg
Error Moving data/images_train/train/Abstract/11456996041128-GAS-Men-Tshirts-5701456996040488-1.jpg
Error Moving data/images_train/train/Abstract/Incynk-Music-Blendz-Everything-Tshirt_c99da3ee23024ec71c275a0cb3592a0c_images.jpg
Error Moving data/images_train/train/Abstract/11470212073422-MTV-Multicoloured-Printed-T-shirt-7961470212073143-1.jpg
Error Moving data/images_train/train/Abstract/11468578337367-Mufti-Men-Tshirts-6981468578336979-1.jpg
Error Moving data/images_train/train/Abstract/11479714418058-IN

  0%|          | 0/318 [00:00<?, ?it/s]

Error Moving data/images_train/train/Colourblocked/11510566380017-John-Miller-Hangout-Men-Tshirts-141510566379864-1.jpg
Error Moving data/images_train/train/Colourblocked/11479288364527-GRITSTONES-Men-Tshirts-9471479288364373-1.jpg
Error Moving data/images_train/train/Colourblocked/11458114716400-John-Players-White--Blue-Striped-Polo-T-shirt-5371458114713917-1.jpg
Error Moving data/images_train/train/Colourblocked/11469170034117-Moda-Rapido-Men-Tshirts-9071469170033862-1.jpg
Error Moving data/images_train/train/Colourblocked/11499253934745-Hypernation-Men-Black-Colourblocked-Cowl-Neck-T-shirt-5971499253934548-1.jpg
Error Moving data/images_train/train/Colourblocked/11466675132110-HRX-by-Hrithik-Roshan-Blue-Active-Training-T-shirt-6131466675131891-1.jpg
Error Moving data/images_train/train/Colourblocked/11464002902364-Monte-Carlo-Boys-Tshirts-4781464002902316-1.jpg
Error Moving data/images_train/train/Colourblocked/11475312073793-Numero-Uno-Men-Tshirts-3401475312073595-1.jpg
Error Movin

  0%|          | 0/1262 [00:00<?, ?it/s]

...

Error Moving data/images_train/train/Typography/11479455041697-Louis-Philippe-Jeans-Men-Grey-Melange-Printed-Round-Neck-T-Shirt-1361479455041356-1.jpg
Error Moving data/images_train/train/Typography/11470740280541-PUMA-Boys-Orange-Printed-DryCELL-T-shirt-3171470740280424-1.jpg
Error Moving data/images_train/train/Typography/11479113879194-Nauti-Nati-Boys-Blue-Printed-Round-Neck-T-Shirt-6341479113879102-1.jpg
Error Moving data/images_train/train/Typography/11496313961462-ONLY-Women-Blue-Printed-Round-Neck-T-shirt-6401496313961258-1.jpg
Error Moving data/images_train/train/Typography/11464870127110-Lawman-pg3-Yellow-Printed-T-shirt-1481464870126703-1.jpg
Error Moving data/images_train/train/Typography/11496907355806-Ed-Hardy-Men-Blue-Self-Design-Round-Neck-T-shirt-6521496907355953-1.jpg
Error Moving data/images_train/train/Typography/11489992994806-JAINISH-Men-Tshirts-5041489992994565-1.jpg
Error Moving data/images_train/train/Typography/11498205176956-Pepe-Jeans-Girls-Peach-Coloured-Sol

 86%|████████▋ | 2251/2609 [00:00<00:00, 3743.22it/s]

Error Moving data/images_train/train/Typography/11505466727135-French-Connection-Men-Blue-Printed-Round-Neck-T-shirt-7901505466727012-1.jpg
Error Moving data/images_train/train/Typography/11473678744353-Moda-Rapido-Men-Black-Printed-Round-Neck-T-Shirt-6081473678744093-1.jpg
Error Moving data/images_train/train/Typography/11503556232542-People-Men-Tshirts-4861503556232432-1.jpg
Error Moving data/images_train/train/Typography/11503397208178-LOCOMOTIVE-Men-Black--White-Printed-Round-Neck-Slim-Fit-T-shirt-6291503397208078-1.jpg
Error Moving data/images_train/train/Typography/11496138935519-RUFF-Boys-Maroon-Printed-Round-Neck-T-shirt-9971496138935421-1.jpg
Error Moving data/images_train/train/Typography/11461912068925-HE-by-MANGO-Beige-Printed-T-shirt-9631461912068752-1.jpg
Error Moving data/images_train/train/Typography/11508852143336-Pepe-Jeans-Women-Peach-Coloured-Printed-Round-Neck-T-shirt-9411508852143200-1.jpg
Error Moving data/images_train/train/Typography/11463049474769-Original-Pen

100%|██████████| 121/121 [00:00<00:00, 3639.32it/s]

Error Moving data/images_train/train/Typography/11491553451399-Lee-Cooper-Girls-Tshirts-3281491553451314-1.jpg
Error Moving data/images_train/train/Typography/11491556788604-Reebok-Black-RE-SS-Polyester-Printed-Running-T-shirt-3431491556788270-1.jpg
Error Moving data/images_train/train/Typography/11511345235895-Reebok-Boys-Tshirts-7921511345235792-1.jpg
Error Moving data/images_train/train/Typography/11469508748352-Nike-Red-Printed-FCB-T-shirt-511469508748093-1.jpg
Error Moving data/images_train/train/Typography/11487579719821-Puma-Men-Charcoal-Printed-Round-Neck-T-Shirt-6661487579719570-1.jpg
Error Moving data/images_train/train/Typography/11458296135175-ROUTE-66-Grey-Melange-Printed-T-shirt-3751458296134529-1.jpg
Error Moving data/images_train/train/Typography/11470746280003-MANGO-Kids-Girls-Off-White-Printed-T-shirt-9491470746279858-1.jpg
Error Moving data/images_train/train/Typography/11470735931680-My-Little-Lambs-Unisex-Tshirts-5521470735931555-1.jpg
Error Moving data/images_trai

In [38]:
import random
a = [1, 2, 3, 4]
random.choices(a, k=2)

[3, 4]

In [35]:
a

[1, 4, 3, 2]

In [3]:
# Delete corrupted Image files
for root, subdir, filenames in os.walk('data/images_train/test/'):
    for fname in tqdm(filenames):
        impath = os.path.join(root, fname)
        try:
            im = Image.open(impath)
        except:
            print('deleted {}'.format(impath))
            os.remove(impath)

  0%|          | 16/14720 [00:00<11:43, 20.89it/s] 

deleted data/images_train/test/1.jpg


 36%|███▌      | 5236/14720 [01:17<02:19, 67.76it/s]

deleted data/images_train/test/aw-16-tee-ocb-24-blue-giraffe-multicolor-10-11-years-original-imaengzbampfqvby.jpeg?dl=1


 76%|███████▌  | 11217/14720 [02:41<00:50, 69.25it/s]

...

 90%|████████▉ | 13221/14720 [03:08<00:21, 70.06it/s]

deleted data/images_train/test/AW-15-DR-13-1.jpg?dl=1
deleted data/images_train/test/aw-16-tee-ocb-08-blue-giraffe-offwhite-ink-2-3-years-original-imaenthx4ughmrpm.jpeg?dl=1


 93%|█████████▎| 13748/14720 [03:16<00:13, 70.08it/s]

deleted data/images_train/test/MR11_XXL.jpg?dl=1


100%|██████████| 14720/14720 [03:29<00:00, 70.24it/s]
